#### Deep Learning Assignment 4(convolutional)

Previously in 2_fullyconnected.ipynb and 3_regularization.ipynb, we trained fully connected networks to classify notMNIST characters.

The goal of this assignment is make the neural network convolutional.


In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import cPickle as pickle
import numpy as np
import tensorflow as tf

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print 'Training set', train_dataset.shape, train_labels.shape
  print 'Validation set', valid_dataset.shape, valid_labels.shape
  print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


#### Reformat into a TensorFlow-friendly shape:

    convolutions need the image data formatted as a cube (width by height by #channels)
    labels as float 1-hot encodings.



In [8]:
#test
num_labels = 10
print(np.arange(num_labels).shape)
print(train_labels[:,None].shape)
print (((np.arange(num_labels) == train_labels[:,None]).astype(np.float32)).shape)

(10,)
(200000, 1, 10)
(200000, 1, 10)


In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32) #4d fm 3d
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32) # 3d but this output is 2d ? check !!
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [9]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [10]:
batch_size = 16 #reduced from 128
patch_size = 5  #reduced from image_size,28
depth = 16
num_hidden = 64 #num_hidden,1024, num_hidden_1( _2):2048

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))  #4d
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))  #2d?
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.  #path_size used be image_size in previous file
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))  #4d  , depth used be num_labels
  layer1_biases = tf.Variable(tf.zeros([depth]))  #depth used be num_labels
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size / 4 * image_size / 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))


In [11]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)


Initialized
Minibatch loss at step 0 : 3.64581
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 50 : 1.52088
Minibatch accuracy: 62.5%
Validation accuracy: 54.4%
Minibatch loss at step 100 : 1.12875
Minibatch accuracy: 68.8%
Validation accuracy: 74.0%
Minibatch loss at step 150 : 0.952278
Minibatch accuracy: 81.2%
Validation accuracy: 77.8%
Minibatch loss at step 200 : 0.419872
Minibatch accuracy: 93.8%
Validation accuracy: 78.4%
Minibatch loss at step 250 : 0.344849
Minibatch accuracy: 93.8%
Validation accuracy: 79.7%
Minibatch loss at step 300 : 0.851721
Minibatch accuracy: 75.0%
Validation accuracy: 79.7%
Minibatch loss at step 350 : 0.862122
Minibatch accuracy: 68.8%
Validation accuracy: 79.7%
Minibatch loss at step 400 : 0.803387
Minibatch accuracy: 81.2%
Validation accuracy: 81.1%
Minibatch loss at step 450 : 0.716209
Minibatch accuracy: 75.0%
Validation accuracy: 80.2%
Minibatch loss at step 500 : 0.902903
Minibatch accuracy: 68.8%
Validation accuracy: 

#### Problem 1

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides a <span style="color:red">max pooling</span> operation (nn.max_pool()) of stride 2 and kernel size 2.

In [12]:
batch_size = 128
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size / 4 * image_size / 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model_pool(data, train=False):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    if train:
      hidden = tf.nn.dropout(hidden, 0.5)

    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model_pool(tf_train_dataset, True)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
   
  # adding regularizers
  regularizers = (tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer1_biases) +
                  tf.nn.l2_loss(layer2_weights) + tf.nn.l2_loss(layer2_biases) +
                  tf.nn.l2_loss(layer3_weights) + tf.nn.l2_loss(layer3_biases) +
                  tf.nn.l2_loss(layer4_weights) + tf.nn.l2_loss(layer4_biases)
                 )
  # Add the regularization term to the loss.
  loss += 3e-4 * regularizers
  
  # Optimizer: set up a variable that's incremented once per batch and
  # controls the learning rate decay.
  batch = tf.Variable(0)
  # Decay once per epoch, using an exponential schedule starting at 0.01.
  learning_rate = tf.train.exponential_decay(
      0.01,                # Base learning rate.
      batch * batch_size,  # Current index into the dataset.
      train_labels.shape[0],          # Decay step.
      0.95,                # Decay rate.
      staircase=True)
  # Use simple momentum for the optimization.
  optimizer = tf.train.MomentumOptimizer(learning_rate,
                                         0.9).minimize(loss,
                                                       global_step=batch)
   
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model_pool(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model_pool(tf_test_dataset))
# In[ ]:

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)


Initialized
Minibatch loss at step 0 : 3.11668
Minibatch accuracy: 10.9%
Validation accuracy: 11.3%
Minibatch loss at step 50 : 2.11099
Minibatch accuracy: 22.7%
Validation accuracy: 51.7%
Minibatch loss at step 100 : 1.4357
Minibatch accuracy: 50.0%
Validation accuracy: 74.8%
Minibatch loss at step 150 : 1.0463
Minibatch accuracy: 68.8%
Validation accuracy: 78.5%
Minibatch loss at step 200 : 1.04754
Minibatch accuracy: 71.9%
Validation accuracy: 80.2%
Minibatch loss at step 250 : 0.967472
Minibatch accuracy: 68.8%
Validation accuracy: 81.0%
Minibatch loss at step 300 : 0.976286
Minibatch accuracy: 71.1%
Validation accuracy: 81.8%
Minibatch loss at step 350 : 0.838838
Minibatch accuracy: 77.3%
Validation accuracy: 82.6%
Minibatch loss at step 400 : 0.89662
Minibatch accuracy: 77.3%
Validation accuracy: 83.4%
Minibatch loss at step 450 : 0.69273
Minibatch accuracy: 85.9%
Validation accuracy: 83.6%
Minibatch loss at step 500 : 0.656299
Minibatch accuracy: 78.9%
Validation accuracy: 83.7%